In [ ]:
# import packages 
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# import PCA-related scikitlearn modules ??
# may just need the modules necessary to turn the PCAs into a multi-linear model


In [ ]:
# import data

# kitchen sink models

gunData = pd.read_excel('firearm_data_cleaned_new.xlsx')
gunData.head()

data1 = gunData[['year', 'state', 'rate', 'deaths', 'law_strength_score',
                    'restrictive_laws', 'permissive_laws', 'total_law_changes',                             
                    'law_strength_change', 'unique_law_classes']].dropna()
    # state is enough, omitting state_name
    # omitting rate_change
data1 = data1.query("state!='District of Columbia'")
    # there is only one datapoint for District of Columbia, flummoxes train_test_split. dropping.


data2 = gunData[['rate', 'state', 'strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']].dropna()
data2 = data2.query("state!='District of Columbia'")

bigX_1 = data1[['year', 'state', 'deaths', 'law_strength_score',
                    'restrictive_laws', 'permissive_laws', 'total_law_changes',                             
                    'law_strength_change', 'unique_law_classes']]
    # state is enough, omitting state_name
    # omitting rate_change

bigX_2 = data2[['state', 'strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']]
bigY1 = data1['rate']
bigY2 = data2['rate']

categories1 = ['year', 'state']
numers1 = ['deaths', 'law_strength_score', 'restrictive_laws', 'permissive_laws', 'total_law_changes',                           
                    'law_strength_change', 'unique_law_classes']

categories2 = ['state']
numers2 = ['strength_background_checks', 'strength_carrying_a_concealed_weapon_ccw', 
                    'strength_castle_doctrine', 'strength_dealer_license', 'strength_firearm_sales_restrictions',
                    'strength_local_laws_preempted_by_state', 'strength_minimum_age','strength_prohibited_possessor',            
                    'strength_registration', 'strength_waiting_period', 'strength_firearm_removal_at_scene_of_domestic_violence',
                    'strength_firearms_in_college_university', 'strength_child_access_laws', 'strength_gun_trafficking', 
                    'strength_open_carry', 'strength_required_reporting_of_lost_or_stolen_firearms',
                    'strength_safety_training_required', 'strength_untraceable_firearms', 'strength_permit_to_purchase',
                    'strength_firearms_in_k_12_educational_settings']

In [ ]:
"""
Apply PCA (IAW notebook 17)

"""

#X = our_data[["sleep_hrs","phone_hrs","outdoor_hrs"]].astype(float)
bigX_1 = bigX_1.astype(float)
bigX_2 = bigX_2.astype(float)
Xc_1 = bigX_1 - bigX_1.mean()
Xc_2 = bigX_2 - bigX_2.mean()

cov_1 = np.cov(Xc_1.T)
cov_2 = np.cov(Xc_2.T)

eigvals_1, eigvecs_1 = np.linalg.eig(cov_1)
eigvals_2, eigvecs_2 = np.linalg.eig(cov_2)

#
#eigvecs

scores = Xc_1.values @ eigvecs_1
scores = Xc_2.values @ eigvecs_2

pc_df_1 = bigX_1.copy()
pc_df_2 = bigX_2.copy()
pc_df_1[["PC1","PC2","PC3"]] = scores
pc_df_2[["PC1","PC2","PC3"]] = scores
#pc_df

loadings_1 = eigvecs_1[:, :2] * np.sqrt(eigvals_1[:2])
loadings_2 = eigvecs_2[:, :2] * np.sqrt(eigvals_2[:2])

loading_df_1 = pd.DataFrame(loadings_1,
                          index=["sleep_hrs", "phone_hrs", "outdoor_hrs"],  # <<<<<<<<<<<<<<<<
                          columns=["PC1_loading", "PC2_loading"])
loading_df_2 = pd.DataFrame(loadings_2,
                          index=["sleep_hrs", "phone_hrs", "outdoor_hrs"],  # <<<<<<<<<<<<<<<<
                          columns=["PC1_loading", "PC2_loading"])